In [2]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "testPwd"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(
        html.A(
            html.Img(src="data:image/png;base64, {}".format(encoded_image.decode()),style={"height": "150px"}),
            href="https://www.snhu.edu"
        ),  
    ),
    html.Center(html.P('Niebla Project 2'),style={"font-size": "0.75em","padding-top": 0}),
    html.Hr(),
    dcc.RadioItems(  
        id="presetFilter",
        options=[
            {"label": "Water Rescue", "value": 1},
            {"label": "Mountain Rescue", "value": 2},
            {"label": "Disaster Rescue", "value": 3},
            {"label": "Reset", "value": 4}
        ],
        value=4,
        labelStyle={"display": "inline-block"},
    ),
#     html.Div(className="row", style={"display": "flex"},children=[
#         html.Button(id="selectButtonOne",n_clicks=0,children="Cats"),
# #         html.Button(id="selectButtonTwo",n_clicks=0,children="Dogs"),
# #         html.Button(id="selectButtonThree",n_clicks=0,children="All")
#         ]
#     ),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
#     html.Div(
#         id='map-id',
#         className='col s12 m6',
#     ),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'), 
    [Input('presetFilter', 'value')]
)
def applyDefaultFilter(value):
    print(value)
    if (value == 4):
        df = pd.DataFrame.from_records(shelter.read({}))
        return df.to_dict('records')
    elif (value == 1):
        df = pd.DataFrame.from_records(shelter.read(
            {"breed": {"$in": ["Labrador Retriever Mix","Pit Bull Mix"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gt" :26,"$lt":156}}
        ))
        return df.to_dict('records')
    elif(value == 2):
        df = pd.DataFrame.from_records(shelter.read(
            {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                "Siberian Husky","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gt" :26,"$lt":156}}
        ))
        return df.to_dict('records')
    elif(value == 3):
        df = pd.DataFrame.from_records(shelter.read(
            {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gt" :20,"$lt":300}}
        ))
        return df.to_dict('records')

    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "derived_viewport_selected_rows")]
)
def update_map(derived_viewport_data, derived_viewport_selected_rows):
#     dff = pd.DataFrame.from_dict(derived_viewport_data)
    dff = df if derived_viewport_selected_rows is None else pd.DataFrame.from_dict(derived_viewport_data)

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[
                dff.iloc[derived_viewport_selected_rows, 13],
                dff.iloc[derived_viewport_selected_rows, 14],
#                 30.75,
#                 -97.48,
            ], 
            children=[
                dl.Tooltip(dff.iloc[derived_viewport_selected_rows, 9]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[derived_viewport_selected_rows,9])
                ])
            ])
        ])
    ]



@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "derived_viewport_selected_rows")]
)
def update_graphs(derived_viewport_data, derived_viewport_selected_rows):
    if(derived_viewport_selected_rows == None):
        derived_viewport_selected_rows =[]
    
    dff = df if derived_viewport_data == None else pd.DataFrame.from_dict(derived_viewport_data)
    # Changing colors on select
    colors = []
    for i in range(len(dff)):
        if(i in derived_viewport_selected_rows):
            colors.append("#ca0b4a")
        else:
            colors.append("#828282")
  
    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [{"y": dff["age_upon_outcome_in_weeks"],"x": dff["name"], "type": "bar", "marker": {"color": colors},}],
                "layout": {
                    "xaxis": {"title": {"text": column}},
                    "height": 500,
                },
            },
        )
        for column in ["age_upon_outcome_in_weeks"]
            if column in dff
    ]


app